In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('/Users/amrutaabhyankar/Downloads/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol='features',
                     labelCol = 'label',
                     predictionCol='prediction')

In [7]:
lrModel = lr.fit(training)

In [8]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

0.14228558260358093

In [11]:
training_summary = lrModel.summary

In [12]:
training_summary.r2

0.027839179518600154

In [13]:
training_summary.rootMeanSquaredError

10.16309157133015

### Splitting the Data into Train and Test Data 

In [14]:
all_data = spark.read.format('libsvm').load('/Users/amrutaabhyankar/Downloads/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt')

In [18]:
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [19]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
|-16.151349351277112|(10,[0,1,2,3,4,5,...|
| -15.86200932757056|(10,[0,1,2,3,4,5,...|
|-15.780685032623301|(10,[0,1,2,3,4,5,...|
| -15.72351561304857|(10,[0,1,2,3,4,5,...|
+----------

In [20]:
test_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 161|
|   mean|-0.11334888737353264|
| stddev|  10.267470276572604|
|    min| -28.571478869743427|
|    max|  20.221961806051706|
+-------+--------------------+



In [21]:
correct_model = lr.fit(train_data)

In [22]:
test_results = correct_model.evaluate(test_data)

In [23]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-27.034043729202125|
| -28.07108469059042|
|-27.944121205986715|
|-22.248535572384398|
|-26.756167243178353|
|-23.295129580663907|
| -20.87492870419196|
|-19.225677352948892|
|-14.515998915577494|
|-18.992366588147647|
| -17.06250939700009|
| -19.04884551283878|
|-15.112710994431126|
|-15.195532616889976|
|-18.264249411330578|
|-15.052713815463619|
| -18.73786819883611|
|-15.036422676313862|
|-15.305792007882735|
|-16.597118113961336|
+-------------------+
only showing top 20 rows



In [24]:
test_results.rootMeanSquaredError

10.400564470189282

In [25]:
unlabeled_data = test_data.select('features')

In [26]:
predictions = correct_model.transform(unlabeled_data)

In [28]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.5374351405413007|
|(10,[0,1,2,3,4,5,...|0.025066652813787005|
|(10,[0,1,2,3,4,5,...|   1.138637777503643|
|(10,[0,1,2,3,4,5,...|  -1.262348517938574|
|(10,[0,1,2,3,4,5,...|  3.8063413069822793|
|(10,[0,1,2,3,4,5,...|   3.083052321705236|
|(10,[0,1,2,3,4,5,...|   0.817446088402746|
|(10,[0,1,2,3,4,5,...|-0.17665867726566187|
|(10,[0,1,2,3,4,5,...| -2.9782014413058495|
|(10,[0,1,2,3,4,5,...|   1.926966962271632|
|(10,[0,1,2,3,4,5,...| 0.03601713279054086|
|(10,[0,1,2,3,4,5,...|   2.787415237386049|
|(10,[0,1,2,3,4,5,...| -0.9729480465903643|
|(10,[0,1,2,3,4,5,...| -0.7559799489045972|
|(10,[0,1,2,3,4,5,...|  2.5321611390913326|
|(10,[0,1,2,3,4,5,...|-0.38467097796759797|
|(10,[0,1,2,3,4,5,...|   3.403100718913771|
|(10,[0,1,2,3,4,5,...|-0.02006029822857...|
|(10,[0,1,2,3,4,5,...| 0.48363909813154515|
|(10,[0,1,2,3,4,5,...|  2.824676